In [1]:
import os
import pandas as pd
from datasketch import MinHash, MinHashLSHEnsemble
from typing import List, Tuple

In [60]:


# --- Configuration ---
BASE_FILE = "D:/uni/AdvancedTopics/project2/sinteticDB/IMDB/IMDB_Base.csv"
NEW_FILE = "D:/uni/AdvancedTopics/project2/sinteticDB/IMDB/Versions/imdb_with_financials_and_directors.csv"
CANDIDATE_DIR = "sinteticDB/IMDB/externalTables"
MAIN_DATASET_KEYS = ["Series_Title"]
NUM_PERM = 128

def create_minhash(values: List[str], num_perm=128):
    m = MinHash(num_perm=num_perm)
    for v in values:
        if pd.notna(v):
            m.update(str(v).strip().lower().encode('utf8'))
    return m

def extract_column_dataframes(directory: str) -> List[Tuple[str, str, List[str], int]]:
    results = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            table_name = filename[:-4]
            df = pd.read_csv(os.path.join(directory, filename))
            for col in df.columns:
                values = df[col].astype(str).dropna().unique().tolist()
                #results.append((table_name, col, values, len(values)))
                results.append((table_name, col, values, len(values),len(df)))
    return results

def get_table(name):
    name=name+".csv"
    table = pd.read_csv(os.path.join(CANDIDATE_DIR, name))
    return table

def recommend_join_type(candidate_keys, main_keys, jaccard_sim, coverage_threshold=0.7):
    key_overlap = len(set(candidate_keys).intersection(set(main_keys)))
    if key_overlap == len(main_keys):
        return "inner join"
    elif key_overlap > 0 and jaccard_sim > coverage_threshold:
        return "left join"
    else:
        return "outer join"

In [61]:
#OPTION 2 reccomend join type
def estimate_join_type_with_confidence(orig_rows, ext_rows, joined_rows):
    results = []
    
    # Inner join likelihood
    if joined_rows <= min(orig_rows, ext_rows):
        confidence = min(90, 100 * (1 - (joined_rows/min(orig_rows, ext_rows))))
        results.append(("inner", confidence))
    
    # Left join likelihood
    if joined_rows == orig_rows:
        results.append(("left", 80))
    elif abs(joined_rows - orig_rows) < orig_rows * 0.1:  # within 10%
        results.append(("left", 60))
    
    # Right join likelihood
    if joined_rows == ext_rows:
        results.append(("right", 80))
    elif abs(joined_rows - ext_rows) < ext_rows * 0.1:  # within 10%
        results.append(("right", 60))
    
    # Full outer likelihood
    if joined_rows >= max(orig_rows, ext_rows):
        expected_min = max(orig_rows, ext_rows)
        expected_max = orig_rows + ext_rows
        if expected_min == expected_max:
            confidence = 0  # tables disjoint
        else:
            confidence = 100 * (1 - abs(joined_rows - (expected_min+expected_max)/2)/(expected_max-expected_min))
        results.append(("full", min(90, confidence)))
    
    if not results:
        return [("unknown", 0)]
    
    # Sort by confidence
    return sorted(results, key=lambda x: -x[1])

In [80]:
def find_joinable_attributes(df1: pd.DataFrame, df2: pd.DataFrame, 
                            sample_size: int = 1000, min_overlap: float = 0.1) -> List[Tuple[str, str]]:
    """
    Find potential joinable attributes between two DataFrames.
    
    Args:
        df1: First DataFrame
        df2: Second DataFrame
        sample_size: Number of rows to sample for value matching (0 for full comparison)
        min_overlap: Minimum ratio of overlapping values to consider columns joinable
    
    Returns:
        List of tuples with matching column pairs (df1_column, df2_column)
    """
    potential_matches = []
    
    # Step 1: Find columns with matching names and compatible dtypes
    common_cols = set(df1.columns) & set(df2.columns)
    for col in common_cols:
        if pd.api.types.is_dtype_equal(df1[col].dtype, df2[col].dtype):
            potential_matches.append((col, col,0))
    
    # Step 2: Find columns with compatible dtypes (even if names don't match)
    for col1 in df1.columns:
        for col2 in df2.columns:
            # Skip if already found or same column pair
            if (col1, col2) in potential_matches or col1 == col2:
                continue
            
            # Check dtype compatibility
            if pd.api.types.is_dtype_equal(df1[col1].dtype, df2[col2].dtype):
                # Get samples (handle empty DataFrames)
                sample1 = _get_sample(df1[col1].dropna(), sample_size)
                sample2 = _get_sample(df2[col2].dropna(), sample_size)
                
                # Skip if either sample is empty
                if len(sample1) == 0 or len(sample2) == 0:
                    continue
                
                # Check for overlapping values
                common_values = set(sample1) & set(sample2)
                if len(common_values) > 0:
                    overlap_ratio = len(common_values) / min(len(sample1.unique()), len(sample2.unique()))
                    if overlap_ratio >= min_overlap:
                        potential_matches.append((col1, col2,overlap_ratio))
        potential_matches.sort(key=lambda x: x[2], reverse=True)
    
    return potential_matches

def _get_sample(series: pd.Series, sample_size: int) -> pd.Series:
    """Helper function to get sample from a series."""
    if sample_size <= 0 or len(series) <= sample_size:
        return series
    return series.sample(sample_size)

In [63]:
# --- Load base and new dataset versions ---
base_df = pd.read_csv(BASE_FILE)
new_df = pd.read_csv(NEW_FILE)


In [64]:
# --- Detect newly added attributes ---
base_cols = set(base_df.columns)
new_cols = set(new_df.columns)
added_cols = new_cols - base_cols

if not added_cols:
    print("✅ No new attributes found.")
    exit()

print(f"🆕 New attributes detected: {', '.join(added_cols)}\n")

🆕 New attributes detected: Director_Awards, Director_Birth_Year, Director_Nationality, Director_Gender, Director_Name, Profitability_Ratio, Release_Season, Production_Budget, Primary_Production_Company, Box_Office_Gross



In [65]:
# --- Index external candidate columns with MinHash + LSH Ensemble ---
column_entries = extract_column_dataframes(CANDIDATE_DIR)
minhashes = []
index_metadata = []

for table_name, col_name, values, size,size_with_na in column_entries:
    mh = create_minhash(values, NUM_PERM)
    minhashes.append(mh)
    index_metadata.append({
        'table': table_name,
        'column': col_name,
        'full_name': f"{table_name}.{col_name}",
        'size': size,
        'keys': [col_name] , 
        'size_with_na':size_with_na
    })

lsh = MinHashLSHEnsemble(threshold=0.1, num_perm=NUM_PERM)
keys = [m['full_name'] for m in index_metadata]
sizes = [m['size'] for m in index_metadata]
combined = list(zip(keys, minhashes, sizes))
lsh.index(combined) 

In [ ]:
#--- For each new attribute: search + recommend join ---
for new_col in added_cols:
    print(f"🔍 Analyzing new attribute: `{new_col}`")

    new_values = new_df[new_col].astype(str).dropna().unique().tolist()
    if not new_values:
        print("⚠️ No values found for this attribute. Skipping.\n")
        continue

    new_attr_minhash = create_minhash(new_values, NUM_PERM)
    candidates=[]
    for key in lsh.query(new_attr_minhash, len(new_values)):
        candidates.append(key)
    
    #rank candidate tables based on jaccard similarity
    ranked = []
    for meta, mh in zip(index_metadata, minhashes):
        if meta['full_name'] in candidates:
            sim = new_attr_minhash.jaccard(mh)
            #join_type = recommend_join_type(meta['keys'], MAIN_DATASET_KEYS, sim)
            join_type =estimate_join_type_with_confidence(len(base_df), len(new_df), meta['size_with_na'])
            join_attribute=find_joinable_attributes(base_df,get_table(meta['table']))
            if join_attribute:
                ranked.append((meta['table'], meta['column'], sim, join_type,join_attribute))

    ranked.sort(key=lambda x: x[2], reverse=True)


    if ranked:
        print("\nTop Candidate Matches:")
        for table, column, sim, join_type,join_attribute in ranked[:5]:
            #print(f" → {table}.{column} | Jaccard: {sim:.4f} | Join: {join_type}")
            print(f" → {table}.{column} | Jaccard: {sim:.4f} | Join: {join_type[0][0]}| Confidence: {join_type[0][1]}")

        best = ranked[0]
        print(f"\n✅ Best match for `{new_col}`: {best[0]}.{best[1]} → obtained trough {best[3][0][0]} join on original_table.{best[4][0][0]} = {best[0]}.{best[4][0][1]} (Jaccard sim: {best[2]:.4f})\n")
    else:
        print("❌ No good matches found for this attribute.\n")

🔍 Analyzing new attribute: `Director_Awards`

Top Candidate Matches:
 → directors.Director_Awards | Jaccard: 1.0000 | Join: left| Confidence: 80
 → movie_actor_bridge.Actor_ID | Jaccard: 0.1094 | Join: inner| Confidence: 37.5

✅ Best match for `Director_Awards`: directors.Director_Awards → obtained trough left join on original_table.Series_Title=directors.Series_Title (Jaccard sim: 1.0000)

🔍 Analyzing new attribute: `Director_Birth_Year`

Top Candidate Matches:
 → directors.Director_Birth_Year | Jaccard: 1.0000 | Join: left| Confidence: 80

✅ Best match for `Director_Birth_Year`: directors.Director_Birth_Year → obtained trough left join on original_table.Series_Title=directors.Series_Title (Jaccard sim: 1.0000)

🔍 Analyzing new attribute: `Director_Nationality`

Top Candidate Matches:
 → directors.Director_Nationality | Jaccard: 1.0000 | Join: left| Confidence: 80
 → country_data.Country | Jaccard: 0.6953 | Join: inner| Confidence: 62.5

✅ Best match for `Director_Nationality`: direct